In [8]:
# Install required packages
# !pip install groq openai python-dotenv tiktoken

import os
import json
from groq import Groq
from typing import List, Dict, Any, Optional
import tiktoken

# Initialize Groq client
api_key = "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"  # Replace with your actual Groq API key
client = Groq(api_key=api_key)

# For token counting (to implement length-based truncation)
encoding = tiktoken.get_encoding("cl100k_base")

class ConversationManager:
    def __init__(self, model: str = "llama-3.1-8b-instant"):
        self.model = model
        self.conversation_history = []
        self.summary = ""
        self.turn_count = 0
        self.summarization_frequency = 3  # Summarize every 3 turns by default

    def add_message(self, role: str, content: str):
        """Add a message to the conversation history"""
        self.conversation_history.append({"role": role, "content": content})
        if role == "user":
            self.turn_count += 1

    def get_conversation_history(self, max_turns: Optional[int] = None, max_tokens: Optional[int] = None) -> List[Dict]:
        """Get conversation history with optional truncation"""
        history = self.conversation_history.copy()
        
        # Apply summary if exists
        if self.summary:
            history = [{"role": "system", "content": f"Summary of previous conversation: {self.summary}"}] + history
        
        # Apply turn-based truncation
        if max_turns and len(history) > max_turns:
            history = history[-max_turns:]
        
        # Apply token-based truncation
        if max_tokens:
            current_tokens = 0
            truncated_history = []
            
            # Process messages in reverse order (keep most recent)
            for message in reversed(history):
                message_tokens = len(encoding.encode(message["content"]))
                if current_tokens + message_tokens <= max_tokens:
                    truncated_history.insert(0, message)
                    current_tokens += message_tokens
                else:
                    break
            
            history = truncated_history
        
        return history

    def summarize_conversation(self) -> str:
        """Generate a summary of the conversation history"""
        if not self.conversation_history:
            return "No conversation to summarize."
        
        # Prepare messages for summarization
        messages = [
            {"role": "system", "content": "You are a helpful assistant that summarizes conversations. Create a concise summary of the key points and decisions from this conversation."},
            {"role": "user", "content": f"Please summarize this conversation:\n\n{self.format_conversation()}"}
        ]
        
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.3,
                max_tokens=500
            )
            summary = response.choices[0].message.content
            self.summary = summary
            return summary
        except Exception as e:
            return f"Error generating summary: {str(e)}"

    def format_conversation(self) -> str:
        """Format conversation history as a string"""
        return "\n".join([f"{msg['role']}: {msg['content']}" for msg in self.conversation_history])

    def chat(self, user_input: str, max_turns: Optional[int] = None, max_tokens: Optional[int] = None) -> str:
        """Process user input and return assistant response"""
        self.add_message("user", user_input)
        
        # Check if it's time to summarize
        if self.turn_count % self.summarization_frequency == 0 and self.turn_count > 0:
            self.summarize_conversation()
        
        # Get truncated history if needed
        messages = self.get_conversation_history(max_turns, max_tokens)
        
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=messages,
                temperature=0.7
            )
            assistant_response = response.choices[0].message.content
            self.add_message("assistant", assistant_response)
            return assistant_response
        except Exception as e:
            return f"Error generating response: {str(e)}"

    def set_summarization_frequency(self, frequency: int):
        """Set how often to summarize (every k turns)"""
        self.summarization_frequency = frequency

    def clear_history(self):
        """Clear conversation history"""
        self.conversation_history = []
        self.summary = ""
        self.turn_count = 0

# Demonstration of Task 1
print("=" * 50)
print("TASK 1: MANAGING CONVERSATION HISTORY WITH SUMMARIZATION")
print("=" * 50)

# Initialize conversation manager
conv_manager = ConversationManager()

# Sample conversation with Indian context
conversation_samples = [
    ("Namaste, I'm Rajesh from Mumbai. I'm looking for a new laptop.", "user"),
    ("Hello Rajesh! I'd be happy to help you find a new laptop. What's your budget in rupees?", "assistant"),
    ("Mera budget around ₹60,000 hai. I need it for coding and sometimes playing games like BGMI.", "user"),
    ("For coding and gaming at that budget, I'd recommend laptops with at least an i5 or Ryzen 5 processor, 8GB RAM (expandable to 16GB), and a dedicated GPU if possible.", "assistant"),
    ("Battery life ka kya scene hai? I need something that can last through my college day.", "user"),
    ("For good battery life, consider laptops with efficient processors. Many modern laptops offer 6-8 hours of battery life. Dell, HP, and Lenovo have good options in this range.", "assistant"),
    ("Any specific models you'd suggest for Indian market?", "user"),
    ("Based on your needs, I'd suggest looking at the Dell Vostro series, Lenovo Ideapad Gaming, or the HP Pavilion series. These are popular in India and good value for money.", "assistant"),
    ("What about after-sales service in Mumbai? Which brand has better service here?", "user"),
    ("In Mumbai, Dell and Lenovo have good service networks. HP also has service centers in most areas. I'd check online reviews for specific service center experiences in your locality.", "assistant"),
    ("Can I upgrade RAM later myself? Should I buy from Amazon or local store like Croma?", "user"),
    ("Many laptops allow RAM upgrades, but check specifications before buying. For purchasing, both Amazon and Croma are good options - compare prices and check for festival discounts or offers.", "assistant")
]

# Add samples to conversation
for content, role in conversation_samples:
    conv_manager.add_message(role, content)

print("Full conversation history:")
print(conv_manager.format_conversation())
print("\n" + "="*50 + "\n")

# Test different truncation settings
print("1. Testing turn-based truncation (last 4 turns):")
truncated = conv_manager.get_conversation_history(max_turns=4)
for msg in truncated:
    print(f"{msg['role']}: {msg['content']}")

print("\n" + "="*50 + "\n")

print("2. Testing token-based truncation (~100 tokens):")
truncated = conv_manager.get_conversation_history(max_tokens=100)
for msg in truncated:
    print(f"{msg['role']}: {msg['content']}")

print("\n" + "="*50 + "\n")

# Test summarization
print("3. Testing conversation summarization:")
summary = conv_manager.summarize_conversation()
print(f"Summary: {summary}")

print("\n" + "="*50 + "\n")

# Test periodic summarization with a new conversation
print("4. Testing periodic summarization (every 3 turns):")
conv_manager.clear_history()
conv_manager.set_summarization_frequency(3)

# Indian travel planning context
test_messages = [
    "Hi, I need help planning a trip to Goa next month.",
    "Sure! Goa is beautiful in December. How many people are going and for how long?",
    "We are 4 friends planning for 5 days. What's the best way to travel from Delhi?",
    "From Delhi, you can fly directly to Goa or take a train like the Goa Express. Flights are faster but more expensive. What's your budget?",
    "We want to keep it under ₹25,000 per person including stay and food.",
    "That's reasonable. Do you prefer beach resorts or something in city area?",
    "We want beachside accommodation but not too expensive.",
    "I'd recommend looking at hotels in Calangute or Baga beach. They have good options at various price points."
]

for i, msg in enumerate(test_messages):
    if i % 2 == 0:  # user messages
        response = conv_manager.chat(msg)
        print(f"User: {msg}")
        print(f"Assistant: {response}")
        
        # Show summary when generated
        if (i//2 + 1) % conv_manager.summarization_frequency == 0:
            print(f"\n[Summary generated after {i//2 + 1} turns]")
            print(f"Current summary: {conv_manager.summary}")
    print("-" * 30)

print("\n" + "="*50 + "\n")

# Task 2: JSON Schema Classification & Information Extraction
print("TASK 2: JSON SCHEMA CLASSIFICATION & INFORMATION EXTRACTION")
print("=" * 50)

# Define the JSON schema for information extraction
user_info_schema = {
    "name": "extract_user_info",
    "description": "Extract user information from the conversation",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {
                "type": "string",
                "description": "The user's name"
            },
            "email": {
                "type": "string",
                "description": "The user's email address"
            },
            "phone": {
                "type": "string",
                "description": "The user's phone number"
            },
            "location": {
                "type": "string",
                "description": "The user's location/city"
            },
            "age": {
                "type": "integer",
                "description": "The user's age"
            }
        },
        "required": []  # All fields are optional as they might not all be present
    }
}

def extract_user_info(text: str) -> Dict[str, Any]:
    """Extract user information using function calling"""
    try:
        response = client.chat.completions.create(
            model="llama-3.1-8b-instant",
            messages=[{"role": "user", "content": text}],
            tools=[{
                "type": "function",
                "function": user_info_schema
            }],
            tool_choice={"type": "function", "function": {"name": "extract_user_info"}},
            temperature=0.1
        )
        
        # Extract the function call arguments
        if response.choices[0].message.tool_calls:
            tool_call = response.choices[0].message.tool_calls[0]
            arguments = json.loads(tool_call.function.arguments)
            return arguments
        else:
            return {"error": "No function call found in response"}
        
    except Exception as e:
        return {"error": str(e)}

# Sample chats for testing with Indian context
sample_chats = [
    "Hi, I'm Priya Sharma. My email is priya.sharma@email.com and I'm 28 years old. I live in Bangalore.",
    "You can reach me at 98765-43210. My name is Rohan Singh and I'm from Delhi.",
    "I'm 35 years old. Contact me at amit.kumar@example.com or call me at 87654-32109. I live in Hyderabad.",
    "Mera naam Suresh Patel hai. Mera phone number hai 76543-21098. Main Ahmedabad mein rehta hoon.",
    "I'm 22 years old. My email is neha.gupta@gmail.com and I'm from Kolkata. My phone is 99887-66554."
]

print("Testing information extraction from sample chats:\n")

for i, chat in enumerate(sample_chats, 1):
    print(f"Sample {i}: {chat}")
    result = extract_user_info(chat)
    print("Extracted information:")
    for key, value in result.items():
        print(f"  {key}: {value}")
    print("\n" + "-"*50 + "\n")

# Validation function
def validate_extraction(extraction: Dict[str, Any]) -> Dict[str, List[str]]:
    """Validate the extracted information against the schema"""
    errors = []
    
    # Check if there was an error in extraction
    if "error" in extraction:
        errors.append(f"Extraction error: {extraction['error']}")
        return {"is_valid": False, "errors": errors}
    
    # Check for unexpected fields
    expected_fields = ["name", "email", "phone", "location", "age"]
    for field in extraction.keys():
        if field not in expected_fields:
            errors.append(f"Unexpected field: {field}")
    
    # Validate email format if present
    if "email" in extraction and extraction["email"]:
        if "@" not in extraction["email"] or "." not in extraction["email"]:
            errors.append(f"Invalid email format: {extraction['email']}")
    
    # Validate age if present - ensure it's an integer
    if "age" in extraction and extraction["age"]:
        try:
            # Convert to integer if it's a string
            if isinstance(extraction["age"], str):
                extraction["age"] = int(extraction["age"])
                
            age = extraction["age"]
            if age < 0 or age > 120:
                errors.append(f"Age out of reasonable range: {age}")
        except (ValueError, TypeError):
            errors.append(f"Age is not a valid integer: {extraction['age']}")
    
    # Validate phone format if present (basic check for Indian numbers)
    if "phone" in extraction and extraction["phone"]:
        phone = extraction["phone"]
        # Remove common punctuation and spaces
        phone_digits = "".join([c for c in phone if c.isdigit()])
        if len(phone_digits) != 10:
            errors.append(f"Indian phone number should have 10 digits: {phone}")
    
    return {"is_valid": len(errors) == 0, "errors": errors}

print("Validating extractions against schema:\n")

for i, chat in enumerate(sample_chats, 1):
    print(f"Sample {i}: {chat}")
    result = extract_user_info(chat)
    validation = validate_extraction(result)
    
    if validation["is_valid"]:
        print("✓ Extraction is valid")
    else:
        print("✗ Extraction has errors:")
        for error in validation["errors"]:
            print(f"  - {error}")
    



TASK 1: MANAGING CONVERSATION HISTORY WITH SUMMARIZATION
Full conversation history:
user: Namaste, I'm Rajesh from Mumbai. I'm looking for a new laptop.
assistant: Hello Rajesh! I'd be happy to help you find a new laptop. What's your budget in rupees?
user: Mera budget around ₹60,000 hai. I need it for coding and sometimes playing games like BGMI.
assistant: For coding and gaming at that budget, I'd recommend laptops with at least an i5 or Ryzen 5 processor, 8GB RAM (expandable to 16GB), and a dedicated GPU if possible.
user: Battery life ka kya scene hai? I need something that can last through my college day.
assistant: For good battery life, consider laptops with efficient processors. Many modern laptops offer 6-8 hours of battery life. Dell, HP, and Lenovo have good options in this range.
user: Any specific models you'd suggest for Indian market?
assistant: Based on your needs, I'd suggest looking at the Dell Vostro series, Lenovo Ideapad Gaming, or the HP Pavilion series. These are

User: We want beachside accommodation but not too expensive.
Assistant: Beachside accommodation in Goa can be a wonderful experience. Here are some options that might fit your budget and preferences:

**Budget-friendly beachside hotels and resorts:**

1. **Arossim Beach:** This beach is located near the town of Cavelossim and offers a range of budget-friendly options, including the Arossim Beach Resort (₹10,000 - ₹15,000 per night) and the Coconut Creek Resort (₹8,000 - ₹12,000 per night).
2. **Benaulim Beach:** This beach is a popular spot for families and offers a range of accommodation options, including the Benaulim Beach Resort (₹9,000 - ₹14,000 per night) and the Varca Beach Resort (₹7,000 - ₹11,000 per night).
3. **Colva Beach:** This beach is a bit more lively than the others and offers a range of accommodation options, including the Colva Beach Resort (₹8,000 - ₹12,000 per night) and the Ocean Palms (₹6,000 - ₹10,000 per night).

**Guesthouses and homestays:**

1. **Cavelossim